# Cognee Graphiti integration demo

First we import the necessary libaries

In [ ]:
import cognee
import logging
import warnings
from cognee.modules.pipelines import Task, run_tasks
from cognee.shared.utils import setup_logging
from cognee.tasks.temporal_awareness import build_graph_with_temporal_awareness
from cognee.infrastructure.databases.relational import (
    create_db_and_tables as create_relational_db_and_tables,
)
from cognee.tasks.temporal_awareness.index_graphiti_objects import (
    index_and_transform_graphiti_nodes_and_edges,
)
from cognee.modules.retrieval.utils.brute_force_triplet_search import brute_force_triplet_search
from cognee.modules.retrieval.graph_completion_retriever import GraphCompletionRetriever
from cognee.infrastructure.llm.prompts import read_query_prompt, render_prompt
from cognee.infrastructure.llm.get_llm_client import get_llm_client

## Set environment variables

In [ ]:
import os

# We ignore warnigns for now
warnings.filterwarnings("ignore")

# API key for cognee
if "LLM_API_KEY" not in os.environ:
    os.environ["LLM_API_KEY"] = ""

# API key for graphiti
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""

GRAPH_DATABASE_PROVIDER = "neo4j"
GRAPH_DATABASE_USERNAME = "neo4j"
GRAPH_DATABASE_PASSWORD = "pleaseletmein"
GRAPH_DATABASE_URL = "bolt://localhost:7687"

os.environ["GRAPH_DATABASE_PROVIDER"] = GRAPH_DATABASE_PROVIDER
os.environ["GRAPH_DATABASE_USERNAME"] = GRAPH_DATABASE_USERNAME
os.environ["GRAPH_DATABASE_PASSWORD"] = GRAPH_DATABASE_PASSWORD
os.environ["GRAPH_DATABASE_URL"] = GRAPH_DATABASE_URL


## Input texts with temporal information

In [ ]:
text_list = [
    "Kamala Harris is the Attorney General of California. She was previously "
    "the district attorney for San Francisco.",
    "As AG, Harris was in office from January 3, 2011 – January 3, 2017",
]

## Running graphiti + transforming its graph into cognee's core system (graph transformation + vector embeddings)

In [ ]:
# 🔧 Setting Up Logging to Suppress Errors
setup_logging(logging.ERROR)  # Keeping logs clean and focused

# 🧹 Pruning Old Data and Metadata
await cognee.prune.prune_data()  # Removing outdated data
await cognee.prune.prune_system(metadata=True)

# 🏗️ Creating Relational Database and Tables
await create_relational_db_and_tables()

# 📚 Adding Text Data to Cognee
for text in text_list:
    await cognee.add(text)

# 🕰️ Building Temporal-Aware Graphs
tasks = [
    Task(build_graph_with_temporal_awareness, text_list=text_list),
]

# 🚀 Running the Task Pipeline
pipeline = run_tasks(tasks)

# 🌟 Processing Pipeline Results
async for result in pipeline:
    print(f"✅ Result Processed: {result}")

# 🔄 Indexing and Transforming Graph Data
await index_and_transform_graphiti_nodes_and_edges()

## Retrieving and generating answer from graphiti graph with cognee retriever

In [ ]:
# Step 1: Formulating the Query 🔍
query = "When was Kamala Harris in office?"

# Step 2: Searching for Relevant Triplets 📊
triplets = await brute_force_triplet_search(
    query=query,
    top_k=3,
    collections=["graphitinode_content", "graphitinode_name", "graphitinode_summary"],
)

# Step 3: Preparing the Context for the LLM
retriever = GraphCompletionRetriever()
context = await retriever.resolve_edges_to_text(triplets)

args = {"question": query, "context": context}

# Step 4: Generating Prompts ✍️
user_prompt = render_prompt("graph_context_for_question.txt", args)
system_prompt = read_query_prompt("answer_simple_question_restricted.txt")

# Step 5: Interacting with the LLM 🤖
llm_client = get_llm_client()
computed_answer = await llm_client.acreate_structured_output(
    text_input=user_prompt,  # Input prompt for the user context
    system_prompt=system_prompt,  # System-level instructions for the model
    response_model=str,
)

# Step 6: Displaying the Computed Answer ✨
print(f"💡 Answer: {computed_answer}")